# Aula 4 - Exercícios - Pandas e Fontes de dados

In [2]:
# Importe as bibliotecas pandas, numpy
import pandas as pd
import numpy as np

### 4.1 Carregue o arquivo do IDEB 2017 no formato de um DataFrame.  Crie uma função para calcular a média do IDEB de 2017 de todos os municípios de um determinado Estado. 
#### Remova os registros cujo valor na coluna REDE sejam Municipal, Estadual ou Federal. Deixe somente os registros cujos valores na coluna REDE sejam Pública.


In [3]:
# Dica 1: Antes de calcular, certifique-se de interpretar os valores '-' como NA durante 
#         o carregamento dos dados ou os substitua por np.nan (NA) com o metodo replace.
# Dica 2: Use a função loc para fazer o filtro de registros.
# Dica 3: Dentre várias maneiras possíveis, uma maneira bastante concisa é usar o operador & (and) para
# congregar dois critérios de filtro (seleção) de registros.
def media_ideb(df, sigla_estado):
    """retorne o valor da media do IDEB de 2017 contido na coluna col do Estado especificado por sigla_estado.
    """
    ### BEGIN SOLUTION
    df.replace({'-': np.nan}, inplace=True)
    bool_estado = df['SG_UF']==sigla_estado
    bool_publica = df['REDE']=='Pública'
    return df.loc[bool_estado & bool_publica, 'IDEB12_17'].mean()
    ### END SOLUTION


In [4]:
## Faça seus testes aqui
path_ideb='https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/ideb/ideb_municipios2017.xlsx'

df_ideb = pd.read_excel(path_ideb, skiprows=9, skipfooter=3, na_values='-')


In [5]:
# Validação
##############################################
########## ATENÇÃO PARA ESTA INSTRUÇÃO #######
# Crie na célula anterior um dataframe df_ideb. 
# Use os argumentos skiprows, na_values e skipfooter para carregar a tabela corretamente.
##############################################
assert round(media_ideb(df_ideb, 'DF'),1) == 3.4
assert round(media_ideb(df_ideb, 'SP'),1) == 4.0
assert round(media_ideb(df_ideb, 'GO'),1) == 4.2

### 4.2 Escreva uma função para fazer o merge dos dataframes de população e de PIB de 2017. A função deve receber como parâmetro o caminho para cada um dos dataframes.

In [6]:
# Dica 1: A coluna que contem o código IBGE do município deve ter o mesmo formato (string) e a mesma
#         quantidade de caracteres nas duas tabelas.
# Dica 2: Converta o codigo do município para string ao carregar o dataframe de população utilizando
#         o mapeamento de tipos dtype={'cod_munic': str, 'cod_uf': str}. Assim o codigo do municipio não perde
#         os zeros a esquerda.
# Dica 3: Concatene o codigo da UF com o código do município usando o operador + e salve em outra coluna.
# Dica 4: Use o código apresentado na aula para carregar o dataframe do PIB de 2017, especifique o dtype da
#         coluna que contém o codigo do municipio.
# Dica 5: Renomeie as colunas apropriadamente antes de fazer o merge. A coluna com o nome do municipio deve se chamar
#         municipio (sem acento) e a coluna com o codigo ibge de 7 caracteres cod_ibge7. Veja o exemplo:
#         {'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 'Unnamed: 2': 'municipio'}. O pandas transformará essas
#         colunas em municipio_x e municipio_y. Mantenha esses dois nomes de coluna, pois eles serão usados no exercício
#         3.7.
# Dica 6: Depois que os dois dataframes tiverem o codigo ibge (com valores equivalentes
#         e mesmo tipo de dados) você pode fazer o merge.

def merge_pop_pib(path_pop, path_pib):
    """retorne um dataframe resultante do merge dos dados do pib e da população
    """
    ### BEGIN SOLUTION
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge7'] = df_pop['cod_uf'] + df_pop['cod_munic']
    df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, dtype={'Unnamed: 1': str})
    df_pib.rename(columns={'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 
                           'Unnamed: 2': 'municipio'}, inplace=True)

    return pd.merge(df_pop, df_pib, on='cod_ibge7')
    ### END SOLUTION


In [7]:
## Faça seus testes aqui
path_pib = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/populacao/estimativa_dou_2017.xlsx'


In [17]:
# Validação
path_pib = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/populacao/estimativa_dou_2017.xlsx'

test_merge = {'pop2017': 25437, '2017': 498864}
df_pib_pop = merge_pop_pib(path_pop, path_pib)
assert df_pib_pop.loc[0,['pop2017', '2017']].to_dict() == test_merge 

### 4.3 Escreva uma função para calcular o PIB percapita de cada município do Brasil, considerando o dataframe do exercício anterior.
#### Nomeie a coluna do PIB Percapita com 'pib_percapita'

In [9]:
def pib_percapita(pib_pop):
    """retorne retorne um dataframe contendo o codigo ibge de 7 caracteres,
       o nome do municipio e seu PIB percapita.
    """
    ### BEGIN SOLUTION
    pib_pop['pib_percapita'] = pib_pop['2017']/pib_pop['pop2017']
    return pib_pop
    ### END SOLUTION


In [10]:
## Faça seus testes aqui



In [11]:
# Validação
# Considere a função merge_pop_pib do exercicio 4.2
df_pib_pop = merge_pop_pib(path_pop, path_pib)

pib_percapita_2017 = pib_percapita(df_pib_pop)
assert round(pib_percapita(df_pib_pop).loc[0,'pib_percapita'],1) == 19.6

### 4.4 Escreva uma função para retornar os registros dos 10 maiores PIBs percapita do brasil (de 2017) a partir do dataframe do exercício anterior. 
#### Escreva uma função genérica que receba como parâmetro um dataframe e o nome da coluna de ordenação.


In [12]:
# Dica 1: Use o método sort_values para ordenar o dataframe. Para uma ordenação descendente use ascending=False.
# Dica 2: Use o atributo iloc para retornar todas as colunas dos 10 primeiros registros.
def top10(df, col):
    """retorne os top 10 registros do dataframe df ordenados por col
    """
    ### BEGIN SOLUTION
    return df.sort_values(by=col, ascending=False).iloc[:10,:]
    ### END SOLUTION


In [13]:
## Faça seus testes aqui
df_pib_pop = merge_pop_pib(path_pop, path_pib)
pib_percapita_2017 = pib_percapita(df_pib_pop)


In [14]:
# Validação
df_pib_pop = merge_pop_pib(path_pop, path_pib)
pib_percapita_2017 = pib_percapita(df_pib_pop)

test_data = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio_x': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8, 3547: 209.3}}
test_data_y = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio_y': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8, 3547: 209.3}}
cidades_0_9 = top10(pib_percapita_2017, 'pib_percapita')[['cod_ibge7', 'municipio_x', 'uf', 'pib_percapita']].iloc[[0,9]]
cidades_0_9_y = top10(pib_percapita_2017, 'pib_percapita')[['cod_ibge7', 'municipio_y', 'uf', 'pib_percapita']].iloc[[0,9]]

cidades_0_9['pib_percapita'] = cidades_0_9['pib_percapita'].apply(lambda x: round(x, 1))
cidades_0_9_y['pib_percapita'] = cidades_0_9_y['pib_percapita'].apply(lambda x: round(x, 1))

assert cidades_0_9.to_dict() == test_data or cidades_0_9_y.to_dict() == test_data_y

### 4.5 Escreva uma função para contar quantos nomes de municípios são diferentes nas bases de dados do PIB e da população. 
#### A coluna do nome dos municípios no dataset de PIB contém também o nome da UF. Remova o nome da UF, por exemplo, removendo os 5 últimos caracteres da string do nome do município, antes de efetuar a comparação.
#### Interprete este resultado. O que você pode extrair de aprendizado deste resultado? 

In [19]:
# Dica 1: use a função map (element-wise) do objeto series, ou seja, sobre a coluna col2 do dataframe df.
# Dica 2: passe uma função lambda para o metodo map que executa as instruções do enunciado.
# Dica 3: compare o resultado da execução da funcao map com a coluna col1
# Dica 4: retorne a quantidade de diferenças resultantes da comparação
def compare_df_cols(df, col1, col2):
    """retorne a quantidade de registros em que o valor do registro na coluna col1 difere do valor em col2
    """
    ### BEGIN SOLUTION
    return (df[col2].map(lambda x: str(x)[:-5]) != df[col1]).sum()
    ### END SOLUTION


In [20]:
## Faça seus testes aqui



In [22]:
# Validação
path_pib = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/populacao/estimativa_dou_2017.xlsx'

df_pib_pop = merge_pop_pib(path_pop, path_pib)
assert compare_df_cols(df_pib_pop, 'municipio_x', 'municipio_y') == 16